# Исследование надежности заемщиков.

**Описание проекта:**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. 1 Обзор данных

In [1]:
import pandas as pd

In [2]:
# откроем таблицу и изучим общую информацию о данных
df = pd.read_csv('/datasets/data.csv')
# выведем первые 10 строк таблицы на экран
display(df.head(10))
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. В столбцах встречаются различные типы данных, а именно:`object`, `int64`, `float64`.

Согласно документации к данным:

*   `children` — количество детей в семье
*	`days_employed` — общий трудовой стаж в днях
*	`dob_years` — возраст клиента в годах
*	`education` — уровень образования клиента
*	`education_id` — идентификатор уровня образования
*	`family_status` — семейное положение
*	`family_status_id` — идентификатор семейного положения
*	`gender` — пол клиента
*	`income_type` — тип занятости
*	`debt` — имел ли задолженность по возврату кредитов
*	`total_income` — ежемесячный доход
*	`purpose` — цель получения кредита

В названиях колонок отсутсвует нарушения стиля.

Однако:
* в колонке с указанием общего трудового стажа в днях `days_employed` встречаются отрицательные значения, кроме того, дни не могут быть вещественным числом. Возможно есть проблема с выгрузкой и про это стоит создать тикет человеку, который эти данные формировал. Но, предварительно, эти данные для анализа нам не нужны.
* в столбце с идентификатором уровня образования `education_id` одни и те же сведния записаны в различном регистре, их необходимо привести к одному виду перед анализом.

Имеет смысл переименовать:

* dob_years в full_years
* children в children_count, все же это не бинарный признак
* education в education_level

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

### Шаг 1.2 Предобработка данных
**Заголовки таблицы**

In [3]:
# переименуем ряд заголовков для более удобной работы с ними
df.set_axis(['children_count', 'days_employed', 'full_years', 'education_level', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose'], axis = 'columns', inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children_count    21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   full_years        21525 non-null  int64  
 3   education_level   21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Один регистр для названий категорий**

In [4]:
# приведем сразу все категории к одному регистру для более удобного анализа
df['education_level'] = df['education_level'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['purpose'] = df['purpose'].str.lower()

# вывод, чтобы проверить результат
display(df.head(10))

,children_count,days_employed,full_years,education_level,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2.1 Заполнение пропусков

In [5]:
# сначала найдем количество пустых значений
print(df.isnull().sum())

children_count         0
days_employed       2174
full_years             0
education_level        0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [6]:
# проверим, какую долю составляют пропущенные значения в каждом из столбцов с пропусками
display(df.isnull().sum()/len(df))

children_count      0.000000
days_employed       0.100999
full_years          0.000000
education_level     0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Таким образом, доля пропущенных занчений в столбцах days_employed и total_income составляет чуть более 10 %. Если говорить о причинах возникновения пропусков, скорее всего они связаны с тем, что у людей нет подтвержденного дохода и стажа работы (неофициальное трудоустройство и т.д). Однако сейчас у нас нет возможности уточнить этот момент у коллег, делающих выгрузку.

In [7]:
# посмотрим, какие именно записи хранят пустые значения
display(df[df['total_income'].isnull()].tail())

,children_count,days_employed,full_years,education_level,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,высшее,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


Среднее и медиана оценивают значения в центре выборки. Однако следует учесть, что медиана оценивает среднее, нивелировав влияние выбросов, которые могут сильно смещать значение среднего. Вот почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

In [8]:
# заполним пропущенные данные total_income
# для заполненния пропущенных значений будем использовать медиану
# чтобы получить более точные данные, используем уровень образования и тип занятости 

# создадим функцию, которая будет вычислять медиану по уровню образования и типу занятости

def calc_median_by_education_and_income(df, education_level, income_type):
    # ориентируемся на оба параметра
    median = df[(df['education_level'] == education_level) & (df['income_type'] == income_type)]['total_income'].median()
    return median

# c помощью последовательной фильтрации с логической индексацией функция 
# сначала получит из исходной таблицы строки с нужным уровнем образования,
# затем из результата отфильтрует строки с нужным типом занатости,
# методом median заполнит пустые значения в колонке  total_income. 
# эти значения функция вернёт в качестве результата
for education_level in df[df['total_income'].isnull()]['education_level'].unique():
    for income_type in df[df['total_income'].isnull()]['income_type'].unique():
        median = calc_median_by_education_and_income(df, education_level, income_type)
        searchRow = (df['education_level'] == education_level) & (df['income_type'] == income_type) & (df['total_income'].isnull())
        df.loc[searchRow, 'total_income'] = df.loc[searchRow, 'total_income'].fillna(median)

# проверка на наличие пустых знанчение в total_income
display(df.isnull().sum())

children_count         0
days_employed       2174
full_years             0
education_level        0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

Столбец с днями трудового стажа сожержит некорретную информацию (это артефакты (аномалии) — значения, которые не отражают действительность или появились по какой-то ошибке). Так количество дней трудового стажа в столбце days_employed в некоторых случаях является отрицательным. Сначала заменим отрицательные занчения столбца на положительные и пропущенные значения на медиану.

In [9]:
# заменим отрицательные занчения столбца на положительные
df['days_employed'] = abs(df['days_employed'])
# проверка результата
display(df.head(10))

,children_count,days_employed,full_years,education_level,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [10]:
# заполним пропущенные данные days_employed
# для заполненния пропущенных значений будем использовать медиану
# чтобы получить более точные данные, используем уровень образования и тип занятости 

# используем ранее созданную функцию, которая будет вычислять медиану
# по уровню образования и типу занятости

# c помощью последовательной фильтрации с логической индексацией функция 
# сначала получит из исходной таблицы строки с нужным уровнем образования,
# затем из результата отфильтрует строки с нужным типом занатости,
# методом median заполнит пустые значения в колонке  days_employed	. 
# эти значения функция вернёт в качестве результата
for education_level in df[df['days_employed'].isnull()]['education_level'].unique():
    for income_type in df[df['days_employed'].isnull()]['income_type'].unique():
        median = calc_median_by_education_and_income(df, education_level, income_type)
        searchRow = (df['education_level'] == education_level) & (df['income_type'] == income_type) & (df['days_employed'].isnull())
        df.loc[searchRow, 'days_employed'] = df.loc[searchRow, 'days_employed'].fillna(median)

# проверка на наличие пустых знанчение в total_income
display(df.isnull().sum())

children_count      0
days_employed       0
full_years          0
education_level     0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Дополнительно изучим данные на уровне подсчета уникальных значений (value_counts()) для каждого столбца, чтобы выявить иные аномалии.

In [11]:
display(df['children_count'].value_counts())# проверка уникальных значений в столбце children_count

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children_count, dtype: int64

In [12]:
display(df['full_years'].value_counts())# проверка уникальных значений в столбце full_years

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: full_years, dtype: int64

In [13]:
display(df['education_level'].value_counts())# проверка уникальных значений в столбце education_level

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_level, dtype: int64

In [14]:
display(df['education_id'].value_counts())# проверка уникальных значений в столбце education_id

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [15]:
display(df['family_status'].value_counts())# проверка уникальных значений в столбце family_status

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [16]:
display(df['family_status_id'].value_counts())# проверка уникальных значений в столбце family_status_id

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [17]:
display(df['gender'].value_counts())# проверка уникальных значений в столбце gender

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [18]:
display(df['income_type'].value_counts())# проверка уникальных значений в income_type 

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [19]:
display(df['debt'].value_counts())# проверка уникальных значений в idebt 

0    19784
1     1741
Name: debt, dtype: int64

In [20]:
display(df['total_income'].value_counts())# проверка уникальных значений в total_income 

136555.108821    820
114842.854099    345
159070.690289    300
165640.744634    237
201785.400018    188
                ... 
101516.604975      1
239154.168013      1
165009.733021      1
94270.049769       1
189255.286637      1
Name: total_income, Length: 19357, dtype: int64

In [21]:
display(df['purpose'].value_counts())# проверка уникальных значений в purpose

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**В результате проведенного анализа установлено следующее:**
* children_count содержит значение 20 и -1, хотя отрицательного числа быть не может и 20 сильно выделяется на фоне остальной выборки
* full_years содержит нулевой возраст
* gender содержит пол XNA
* purpose содержит опечатку - ремонт жильЮ

**Возможные причины выявленных аномалий:**
* значение 20 в количестве детей скорее всего опечатка, когда набирая 2, случайно задели ноль.
* значение -1 могло означать раннее отсуствие детей либо их наличие, но почеуму-то выгруженное, как отрицательное число. Так как по df[df['children_count'] == -1]['family_status'].value_counts() можно убедиться, что большая часть этих людей находится или находилась в браке, остановимся на втором варианте.
* возраст нулевым быть не может, скорее всего он нам просто неизвестен. Так как для целью нашего исследования вопрос о влиянии возраста на возврат кредита в срок  в данный момент не является, исправлять значения не будем.
* поскольку пол XNA только один и невозможно достоверно сказать, к какому полу ближе это значение, то оставим его без изменений.
Исправим эти моменты.

In [22]:
# исправим значения 20 для children_count
df.loc[df['children_count'] == 20, 'children_count'] = 2
# проверка результата
print(df.loc[df['children_count'] == 20]['children_count'].count())

0


In [23]:
# исправим значение -1 для количества детей согласно нашей гипотезы (то есть заменим на 1)
df.loc[df['children_count'] == -1, 'children_count'] = 1
# проверка результата
print(df.loc[df['children_count'] == -1]['children_count'].count())

0


In [24]:
# исправим опечатку в purpose
df.loc[df['purpose'] == 'ремонт жилью', 'purpose'] = 'ремонт жилья'
# проверка результата
print(df.loc[df['purpose'] == 'ремонт жилью']['purpose'].count())

0


### Шаг 2.3. Изменение типов данных.

In [25]:
# заменим тип total_income на int
# поскольку нам нужно конкретный целочисленный тип, а не вещественный, используем именно astype
df['total_income'] = df['total_income'].astype('int')

# проверка результата
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children_count    21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   full_years        21525 non-null  int64  
 3   education_level   21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

In [26]:
# проверим, есть ли дубликаты в таблице с помощью метода duplicated()
# использование именно этого метода обусловлено тем, что проверка на наличие дубликатов с помощью  value_counts()
# потребует создания отдельной Series, которая в дальнейшем не пригодится
print(df.duplicated().sum())

71


In [27]:
df = df.drop_duplicates().reset_index(drop=True)# удаление явных дубликатов (с удалением старых индексов и формированием новых)
print(df.duplicated().sum())# проверка на отсутствие дубликатов

0


**Вывод**

Дубликатов оказалось не так много.Возможно, это были одобренные заявки от одних и тех же людей либо просто человеческий фактор.
От неявных дубликатов в колонке education_level мы избавились ранее в шаге 1.2, когда привели сразу все категории к одному регистру для более удобного анализа.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [28]:
#создадим новый датафрейм со столбцами:
#education_id и education_level
education_log = df[['education_id','education_level']]
# удалим дубликаты из словаря
education_log = education_log.drop_duplicates().reset_index(drop=True)
#проверка результата
display(education_log.head(10))

,education_id,education_level
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [29]:
#создадим новый датафрейм со столбцами:
#family_status_id и family_status
family_log = df[['family_status_id','family_status']]
# удалим дубликаты из словаря
family_log = family_log.drop_duplicates().reset_index(drop=True)
#проверка результата
display(family_log.head(10))

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [30]:
#удалим из исходного датафрейма столбцы education_level и family_status, оставив только их идентификаторы:
#education_id и family_status_id:
del df['education_level']
del df['family_status']
#проверка результата
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children_count    21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   full_years        21454 non-null  int64  
 3   education_id      21454 non-null  int64  
 4   family_status_id  21454 non-null  int64  
 5   gender            21454 non-null  object 
 6   income_type       21454 non-null  object 
 7   debt              21454 non-null  int64  
 8   total_income      21454 non-null  int64  
 9   purpose           21454 non-null  object 
dtypes: float64(1), int64(6), object(3)
memory usage: 1.6+ MB
None


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями дохода:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.


In [31]:
#создадим функцию, которая соотносит уровень дохода клиента с опредленной категорией 
def make_income_category(total_income):
    if total_income < 30000:
        return 'E'
    if 30001 <= total_income < 50000:
        return 'D'
    if 50001 <= total_income < 200000:
        return 'C'
    if 200001 <= total_income < 1000000:
        return 'B'
    return 'A'

#создадим столбец total_income_category, в котором сохраним результатработы функции
df['total_income_category'] = df['total_income'].apply(make_income_category) 

# проверка результата
print(df['total_income_category'].value_counts())

C    15829
B     5228
D      350
A       25
E       22
Name: total_income_category, dtype: int64


### Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
* операции с автомобилем,
* операции с недвижимостью,
* получение образования.


In [32]:
# создадим списки с ункиальными занчениями столбца purpose (часть строки, не полное наименование)
# для дальнешей обработки функцией
target_car = ['автомобиль', 'автомобиля', 'автомобилем', 'автомобили']
target_realty = ['недвижимость', 'недвижимости', 'недвижимостью', 'жилье', 'жилья', 'жильем']
target_wedding = ['сваьба', 'свадьбы', 'свадьбу']
target_education = ['образование', 'образования', 'образованием'] 

def make_purpose_category(purpose):
    for value in target_car:
        if value in  purpose:
            return 'операции с автомобилем'
            
    for value in target_realty:
         if value in  purpose:
            return 'операции с недвижимостью'
        
    for value in target_wedding:
         if value in  purpose:
            return 'проведение свадьбы'
            
    for value in target_education:
        if value in  purpose:
            return 'получение образования'
            
    return 'иные цели'

#создадим столбец purpose_category, в котором сохраним результатработы функции
df['purpose_category'] = df['purpose'].apply(make_purpose_category) 

#проверка результата
display(df['purpose_category'].value_counts())

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           1533
иные цели                     791
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1:  Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на поставленный вопрос сначала создадим столбец children_category со ледующими категориями:
* нет детей
* 1 ребенок;
* 2 ребенка;
* много детей - если в семье более 2-х детей

In [33]:
#создадим функцию, которая соотносит количество детей клиента с опредленной категорией 
def make_children_category(children_count):
    if children_count == 1:
        return '1 ребенок'
    if children_count == 2:
        return '2 ребенка'
    if children_count >= 3:
        return 'много детей'
    return 'нет детей'

df['children_category'] = df['children_count'].apply(make_children_category)

# проверим, что все ок
display(df['children_category'].value_counts())

нет детей      14091
1 ребенок       4855
2 ребенка       2128
много детей      380
Name: children_category, dtype: int64

In [34]:
# теперь создадим функцию, которая позволить рассчитывать отношение в процентах для ответов на все вопросы исследоваания
def make_proportion(pdSerises):
    return str(round((pdSerises.sum() / pdSerises.count()) * 100, 2)) + '%'
# построим сводную таблицу
data_pivot = df.pivot_table(index=['children_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем полученный результат
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
display(data_pivot)

,sum,count,make_proportion
,debt,debt,debt
children_category,,,
нет детей,1063,14091,7.54%
много детей,31,380,8.16%
1 ребенок,445,4855,9.17%
2 ребенка,202,2128,9.49%


##### Вывод 1:

Таким образом, наибольшую вероятность возниконовения задолженности имеют клиенты с 2 детьми. При этом процент задолженности в семьях без детей и в семьях, где количество детей более 2-х примерно одинаков. Полученные данные свидетельствуют о том, что существует зависимость между количеством детей и возвратом кредита в срок.

##### Вопрос 2:  Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
# поскольку мы до этого удаляли из основных данных  family_status, тут необходимо их вернуть
df_with_family_status = df.merge(family_log, on='family_status_id', how='left')

# построим сводную таблицу
data_pivot = df_with_family_status.pivot_table(index=['family_status'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем полученный результат
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
display(data_pivot)

,sum,count,make_proportion
,debt,debt,debt
family_status,,,
вдовец / вдова,63,959,6.57%
в разводе,85,1195,7.11%
женат / замужем,931,12339,7.55%
гражданский брак,388,4151,9.35%
не женат / не замужем,274,2810,9.75%


##### Вывод 2:

Согласно полученным данным вдовец/вдова и люди в разводе имеют наименьший процент возникновения задолженности. Наибольшая вероятность просрочки по кредиту имеют неженатые и состоящие в гражданском браке.  Полученные данные свидетельствуют о том, что существует зависимость между семейным положением и возвратом кредита в срок.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
# построим сводную таблицу
data_pivot = df.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем полученный результат
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
display(data_pivot)

,sum,count,make_proportion
,debt,debt,debt
total_income_category,,,
D,21,350,6.0%
B,364,5228,6.96%
A,2,25,8.0%
C,1352,15829,8.54%
E,2,22,9.09%


##### Вывод 3:

Как и следовало ожтдать,  невысокий процент просрочки у людей со средним доходом (категория D), и доходом выше среднего (категория B), а частые просрочки по кредитам у людей снизким уровнем дохода. 

##### Вопрос 4:  Как разные цели кредита влияют на его возврат в срок?

In [37]:
# построим сводную таблицу 
data_pivot = df.pivot_table(index=['purpose_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем полученный результат
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
display(data_pivot)

,sum,count,make_proportion
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,782,10811,7.23%
проведение свадьбы,122,1533,7.96%
иные цели,64,791,8.09%
получение образования,370,4013,9.22%
операции с автомобилем,403,4306,9.36%


##### Вывод 4:

Полученные данные свидетельствуют о том, что выгоднее всего давать кредиты на недвижимость, а автомобиль и образование - в зоне риска. 

### Общий вывод:

Самый идеальный заемщик - это человек без детей, состоящий или состоявший в официальных отношениях, имеющий средний или выше среднего уровень дохода и берущий деньги на недвижимость.
Разница в процентном соотношении просроченных выплат по каждой из категорий примерно одинаковая. В среднем, это диапазоны от 6 до 9 %, то есть относительная разница составляет в среднем 3 %, что означает разницу в вероятности просрочки платежа примерно 30%.